In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets,layers,models
import matplotlib.pyplot as plt

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the dataset paths
base_dir = "archive/wheat_leaf"  # Replace with the path to your data
categories = ["Healthy", "septoria", "stripe_rust"]

# Create folders for splits
splits = ["train", "validation", "test"]
for split in splits:
    for category in categories:
        os.makedirs(os.path.join(base_dir, split, category), exist_ok=True)

# Split data
for category in categories:
    category_dir = os.path.join(base_dir, category)
    images = os.listdir(category_dir)

    # Train-test split
    train_images, temp_images = train_test_split(images, test_size=0.4, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

    # Define split directories
    split_dirs = {
        "train": train_images,
        "validation": val_images,
        "test": test_images
    }

    # Move images to respective folders
    for split, image_list in split_dirs.items():
        for image_name in image_list:
            src = os.path.join(category_dir, image_name)
            dst = os.path.join(base_dir, split, category, image_name)
            shutil.move(src, dst)

print("Dataset successfully divided into train, validation, and test sets!")


Dataset successfully divided into train, validation, and test sets!


In [2]:
import splitfolders #used for splitting the files
splitfolders.ratio("wheat_leaf", output= "splits for wheat",seed=1337, ratio = (.8,.1,.1),group_prefix=None,move=False)

Copying files: 407 files [00:07, 54.98 files/s]


In [3]:
#load the datasets
import keras

train_ds = keras.utils.image_dataset_from_directory(
    directory = "splits_for_wheat/train",
    labels = "inferred",
    label_mode = 'categorical',
    batch_size = 32,
    image_size = (256,256)
)
val_ds = keras.utils.image_dataset_from_directory(
    directory = "splits_for_wheat/val",
    labels = "inferred",
    label_mode = 'categorical',
    batch_size = 32,
    image_size = (256,256)
)

Found 324 files belonging to 3 classes.
Found 39 files belonging to 3 classes.


In [4]:
model = keras.applications.Xception(
    weights=None, input_shape=(256, 256, 3), classes=3)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_ds, epochs=10, validation_data=val_ds,)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 232s 14s/step - accuracy: 0.4021 - loss: 2.7573 - val_accuracy: 0.5128 - val_loss: 1.0970
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 118s 11s/step - accuracy: 0.5954 - loss: 1.3122 - val_accuracy: 0.5128 - val_loss: 1.0924
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 127s 11s/step - accuracy: 0.5451 - loss: 0.9149 - val_accuracy: 0.5128 - val_loss: 1.0794
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 131s 12s/step - accuracy: 0.7138 - loss: 0.7429 - val_accuracy: 0.5128 - val_loss: 1.0660
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 141s 12s/step - accuracy: 0.7134 - loss: 0.6054 - val_accuracy: 0.5128 - val_loss: 1.0582
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 131s 12s/step - accuracy: 0.8225 - loss: 0.4949 - val_accuracy: 0.5128 - val_loss: 1.0377
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 128s 11s/step - accuracy: 0.7850 - loss: 0.5604 - val_accuracy: 0.5128 - val_loss: 1.0471
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 123s 11s/step - accuracy: 0.7720 - loss: 0.7307 - val_accuracy: 0.

In [14]:
model.save("whaet_Xception_model.keras")

## train a cnn 

In [20]:
#create a cnn model
model = models.Sequential()
model.add(layers.Conv2D(256,(3, 3), activation='relu' ,input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

"""
model.add(layers.Conv2D(256,(3,3), activation ='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3,3), activation= 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
"""

In [21]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 254, 254, 256)  │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 127, 127, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 125, 125, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 62, 62, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 60, 60, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,777,408 (6.78 MB)

 Trainable params: 1,777,408 (6.78 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='softmax' ))
model.add(layers.Dense(3))

In [23]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 254, 254, 256)  │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 127, 127, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 125, 125, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 62, 62, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 60, 60, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │     1,605,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,383,171 (12.91 MB)

 Trainable params: 3,383,171 (12.91 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
#compile the model
model.compile(optimizer='adam',
               loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

history = model.fit(train_ds,epochs=15,
                    validation_data=val_ds)

Epoch 1/15


11/11 ━━━━━━━━━━━━━━━━━━━━ 149s 13s/step - accuracy: 0.4089 - loss: 1.1137 - val_accuracy: 0.5128 - val_loss: 1.0569
Epoch 2/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 165s 15s/step - accuracy: 0.5343 - loss: 1.0533 - val_accuracy: 0.5128 - val_loss: 1.0531
Epoch 3/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 167s 15s/step - accuracy: 0.5179 - loss: 1.0507 - val_accuracy: 0.5128 - val_loss: 1.0505
Epoch 4/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 161s 14s/step - accuracy: 0.5094 - loss: 1.0494 - val_accuracy: 0.5128 - val_loss: 1.0483
Epoch 5/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 164s 15s/step - accuracy: 0.4927 - loss: 1.0524 - val_accuracy: 0.5128 - val_loss: 1.0463
Epoch 6/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 163s 14s/step - accuracy: 0.5282 - loss: 1.0367 - val_accuracy: 0.5128 - val_loss: 1.0444
Epoch 7/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 165s 15s/step - accuracy: 0.5086 - loss: 1.0431 - val_accuracy: 0.5128 - val_loss: 1.0426
Epoch 8/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 171s 15s/step - accuracy: 0.5395 - loss: 1.0334 - val_accuracy: 0.5128 - val_